In [11]:
import requests
from bs4 import BeautifulSoup 
import csv
from itertools import zip_longest 
import pandas as pd 
from tqdm import tqdm
import time 

In [12]:
Company_caegory_links=[]
Company_city_links=[]
company_name=[]
company_category=[]
company_city=[]
company_activity=[]
company_phone=[]
company_link=[]

In [13]:
main_link ="https://www.egyptianindustry.com/"

# finding all categorys and city links

In [14]:
def find_categorys_city(main_link):
  Company_caegory_links=[]
  Company_city_links=[]
  request = requests.get(main_link)
  src =request.content
  soup =BeautifulSoup(src ,"lxml")
  Company_city_soup=soup.find("div",{"id":"demozones"}).find("section").find("div").find_all("p")
  Company_category_soup=soup.find("div",{"id":"demo"}).find("section").find("div").find_all("p")

  for i in tqdm(range(len(Company_city_soup))):
      Company_city_links.append(Company_city_soup[i].find('a')['href'])
  for i in tqdm(range(len(Company_category_soup))):
      Company_caegory_links.append(Company_category_soup[i].find('a')['href'])
  return Company_caegory_links ,Company_city_links

In [15]:
Company_caegory_links ,Company_city_links=find_categorys_city(main_link)

100%|██████████| 31/31 [00:00<00:00, 32449.07it/s]


In [6]:
len(Company_caegory_links)

31

In [7]:
# collect data with city area 
for t in tqdm(range(0,len(Company_city_links))):
  request = requests.get(Company_city_links[t])
  src =request.content
  soup =BeautifulSoup(src ,"lxml")
  try:
    citys_length =int(soup.find("li",{"id":"news_next"}).find_all("a")[1]["href"].split("=")[1])
  except:
    citys_length=1
  for j in range(1,citys_length+1):
    request = requests.get(Company_city_links[t]+"?page="+str(j))
    src =request.content
    soup =BeautifulSoup(src ,"lxml")
    names=soup.find_all("h1",{"class":"f-listings-item__title"})
    phones=soup.find_all("address",{"class":"f-listings-item__address"})
    category=soup.find_all("ul",{"class":"wil-icon-list"})
    for i in range(len(names)):
        company_name.append(names[i].text)
        company_link.append(names[i].find("a")["href"])
        company_phone.append(phones[i].text[1:-1].split("\n")[0])
        try:
          company_activity.append(phones[i].text[1:-1].split("\n")[1])
        except:
          company_activity.append(" غير محددة ")
        company_category.append(category[i].find("li").find("span").find("h2").text[1:].split("\n")[1].strip())
        company_city.append(category[i].find("li").find("span").find("h2").text[1:].split("\n")[0][:-3].strip())

100%|██████████| 60/60 [14:49<00:00, 14.83s/it]


In [16]:
len(company_activity)#11522

0

In [17]:
# collect data with Company_caegorys area 
for t in tqdm(range(0,len(Company_caegory_links))):
  request = requests.get(Company_caegory_links[t])
  src =request.content
  soup =BeautifulSoup(src ,"lxml")
  try:
    citys_length =int(soup.find("li",{"id":"news_next"}).find_all("a")[1]["href"].split("=")[1])
  except:
    citys_length=1
  for j in range(1,citys_length+1):
    request = requests.get(Company_caegory_links[t]+"?page="+str(j))
    src =request.content
    soup =BeautifulSoup(src ,"lxml")
    names=soup.find_all("h1",{"class":"f-listings-item__title"})
    phones=soup.find_all("address",{"class":"f-listings-item__address"})
    category=soup.find_all("ul",{"class":"wil-icon-list"})
    for i in range(len(names)):
        company_name.append(names[i].text)
        company_link.append(names[i].find("a")["href"])
        company_phone.append(phones[i].text[1:-1].split("\n")[0])
        try:
          company_activity.append(phones[i].text[1:-1].split("\n")[1])
        except:
          company_activity.append(" غير محددة ")
        company_category.append(category[i].find("li").find("span").find("h2").text[1:].split("\n")[1].strip())
        company_city.append(category[i].find("li").find("span").find("h2").text[1:].split("\n")[0][:-3].strip())

100%|██████████| 31/31 [15:50<00:00, 30.65s/it]


In [18]:
df=pd.DataFrame({
    "company name":company_name,
    "company phone":company_phone,
    "company category":company_category,
    "company city":company_city,
    "company activity":company_activity,
    "company link":company_link
})
df

,company name,company phone,company category,company city,company activity,company link
0,شركة ثرى إم للصناعات المعدنية,هاتف المصنع: 0663758105 - 0663758106,تشغيل المعادن,مدينة بورسعيد,النشاط: تصنيع الهناجر (الجمالون) والخزانات ...,https://www.egyptianindustry.com/SearchR/1/Pag...
1,شركة ريدى فينس للأسوار الجاهزة والملاعب | مبان...,الموبيل: 01001112523 - 01116665696,تشغيل المعادن,مدينة السادس من أكتوبر,النشاط: مبانى جاهزة - أسوار سابقة الصنع - ل...,https://www.egyptianindustry.com/SearchR/1/Pag...
2,مصنع المسلمى للأبواب الأوتوماتيكية | أبواب رول...,هاتف المصنع: 0554334033 - 0554334030,تشغيل المعادن,مدينة العاشر من رمضان,النشاط: أبواب رول - أبواب شتر - أبواب عالية...,https://www.egyptianindustry.com/SearchR/1/Pag...
3,الشركة الأهلية للتصنيع | كبسولات ألومنيوم - أغ...,هاتف المصنع: 0554410285,تشغيل المعادن,مدينة العاشر من رمضان,النشاط: كبسولات ألومنيوم - أغطية أدوية - أغ...,https://www.egyptianindustry.com/SearchR/1/Pag...
4,الشركة العالمية تكنو ميتال | وحدات تخزين معدن ...,الموبيل: 01000846938,تشغيل المعادن,مدينة العاشر من رمضان,النشاط: وحدات تخزين معدن - دواليب معدن,https://www.egyptianindustry.com/SearchR/1/Pag...
...,...,...,...,...,...,...
11542,مصنع جاز مان لتعبئة إسطوانات البوتاجاز | تعبئة...,الموبيل: 01000399023,البترول والتعدين,مدينة الدقهلية,النشاط: تعبئة إسطوانات البوتاجاز,https://www.egyptianindustry.com/SearchR/1/Pag...
11543,مصنع جذور لإنتاج وتصدير الفحم النباتى | فحم ما...,الموبيل: 01005554860,البترول والتعدين,مدينة دمياط,النشاط: فحم مانجو - فحم ليمون - فحم برتقال ...,https://www.egyptianindustry.com/SearchR/1/Pag...
11544,مصنع سورس إيجيبت | خامات تعدينية,الموبيل: 01009799906,البترول والتعدين,مدينة المنيا,النشاط: خامات تعدينية,https://www.egyptianindustry.com/SearchR/1/Pag...
11545,مصنع وايت ستار أويل سلفنت | خطوط إنتاج زيوت - ...,هاتف المصنع: 033424093,البترول والتعدين,مدينة الإسكندرية,النشاط: خطوط إنتاج زيوت - خطوط إنتاج مزيلات...,https://www.egyptianindustry.com/SearchR/1/Pag...


In [19]:
df.to_excel(" دليل الصناعات المصرية نسخة القطاعات الصناعية .xlsx")